In [22]:
from transformers import AutoTokenizer, RobertaForQuestionAnswering
from datasets import load_dataset
import torch

In [23]:
tokenizer = AutoTokenizer.from_pretrained("deepset/roberta-base-squad2")
model = RobertaForQuestionAnswering.from_pretrained("deepset/roberta-base-squad2")

In [76]:
train_data = load_dataset("csv", data_files="Dados/train_data.csv")
test_data = load_dataset("csv", data_files="Dados/test_data.csv")

print(train_data['train'])

Dataset({
    features: ['context', 'question', 'answer', 'start'],
    num_rows: 41
})


In [113]:
def tokenize_function(example):
    context_prompt = 'context: '
    question_prompt = ' question: '
    answer_human_prompt = ' human_answer: '
    answer_model_prompt = ' model_answer: '
    prompts = [context_prompt + context + question_prompt + question + answer_human_prompt for context, question in zip(example["context"], example["question"])]
    respostas = [answer + answer_model_prompt for answer in example["answer"]]
    inputs = tokenizer(prompts, padding="max_length", truncation=True, max_length = 1000, return_tensors="pt")
    labels = tokenizer(respostas, padding="max_length", truncation=True, max_length = 1000, return_tensors="pt")
    combined_input_ids = torch.cat((inputs.input_ids, labels.input_ids), dim=1)
    example['input_ids'] = combined_input_ids
    example['attention_mask'] = inputs.attention_mask

    return example

tokenized_train_data = train_data.map(tokenize_function, batched=True)
tokenized_test_data = train_data.map(tokenize_function, batched=True)

Map:   0%|          | 0/41 [00:00<?, ? examples/s]

In [114]:
tokenizer.decode(tokenized_test_data['train'][5]['input_ids'], skip_special_tokens=True)

'context: O Aprendiz do Futuro é uma iniciativa do Governo do Estado de Goiás, por meio da Secretaria de Estado de Desenvolvimento Social (SEDS), que tem como visão oferecer o maior programa de sócio-aprendizagem do Brasil e do mundo com foco em tecnologia, alta performance e impacto social na vida dos jovens em situação de vulnerabilidade. question: Quais a visão do programa Aprendiz do Futuro? human_answer: A visão do Programa Aprendiz do Futuro é criar um ambiente de aprendizado inovador e inclusivo, onde os jovens possam desenvolver suas habilidades e alcançar seu pleno potencial, contribuindo assim para um futuro melhor para si e para suas comunidades. model_answer: '

In [ ]:
output_dir = f'./question-answering-training-{str(int(time.time()))}'

training_args = TrainingArguments(
    output_dir=output_dir,
    learning_rate=1e-5,
    num_train_epochs=1,
    weight_decay=0.01,
    logging_steps=1,
    max_steps=1
)

trainer = Trainer(
    model=original_model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation']
)

In [ ]:
trainer.train()